## Assignment 1
The goal of this assignment is to write a simple back-testing algorithm. It is not a test for you to come up with new strategies. 
I have laid out all the instructions. ChatGPT is the best friend in coding. 
P.S. if this is too easy, happy to give your more difficult tasks:-) 

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import matplotlib.pyplot as plt

#### Read in NYSE.txt into a DataFrame, which includes stock tickers and names


In [3]:
#read the stock tickers and names into a DataFrame
df = pd.read_table('NYSE.txt')

#Create a list contains all tickers: iterate through stock list and append tickers into an empty list
tickers = list(df['Symbol'])

In [4]:
df

,Symbol,Description
0,A,Agilent Technologies
1,AA,Alcoa Corporation
2,AAC,Aac Holdings Inc
3,AAN,Aaron's Inc
4,AAP,Advance Auto Parts Inc
...,...,...
3154,ZPIN,Zhaopin Ltd
3155,ZTO,Zto Express [Cayman] Inc
3156,ZTR,Zweig Total Return Fund
3157,ZTS,Zoetis Inc Class A Common Stoc


#### Using one stock as an example to construct buy-sell strategy 

Step 1: Download data and calculate necessary summary statistics

In [5]:
#download stock data for the FIRST stock and place in DataFrame (using yf.download)
data1 = yf.download('A', start='2018-01-01')

[*********************100%%**********************]  1 of 1 completed


In [6]:
#create column to hold our 90 day rolling standard deviation 
data1['90_Day_Rolling'] = data1['Close'].rolling(90).std()

In [7]:
#create a column to hold our 20 day moving average 
data1['20_Day_Mean'] = data1['Close'].rolling(20).mean()

Step 2: Create "BUY" signal according to two conditions

In [8]:
#BUY Condition 1: create a column which holds a TRUE value if the gap down from previous day's low to next day's open is larger than the 90 day rolling HALF of standard deviation
data1['Buy_1'] = (data1['Low'].shift(-1) - data1['Open']) > (data1['90_Day_Rolling']/2)

In [9]:
#BUY Condition 2: create a column which holds a TRUE value if the opening price of the stock is above the 20 day moving average 
data1['Buy_2'] = data1['Open'] > data1['20_Day_Mean']

In [10]:
#"BUY" signal: create a column that holds a TRUE value if both buy criteria are also TRUE
data1['Buy_Signal'] = data1['Buy_1'] & data1['Buy_2']

Step 3: Create "SELL" signal according to two conditions

In [11]:
#SELL Condition 1: create a column which holds a TRUE value if the gap up from previous day's high to next day's open is larger than the 90 day rolling HALF of standard deviation
data1['Sell_1'] = (data1['High'].shift(-1) - data1['Open']) > (data1['90_Day_Rolling']/2)

In [12]:
#SELL Condition 2: create a column which holds a TRUE value if the opening price of the stock is below the 20 day moving average 
data1['Sell_2'] = data1['Open'] < data1['20_Day_Mean']

In [13]:
#"SELL" signal: create a column that holds a TRUE value if both sell criteria are also TRUE
data1['Sell_Signal'] = data1['Sell_1'] & data1['Sell_2']

Step 4: Show the results of Trading Algo

In [14]:
#calculate daily % return series for stock
data1['Daily_Return'] = data1['Close'].pct_change()

##### A Chanllenge: create an indicator which equals to 1 if you hold one share and to -1 if you sell one share. 
As a example, if at day 10, the algo tells you to buy a share, and at day 15, the algo tells you to sell a share, then you do not hold any share from day 1 to day 10, and hold one share form day 11 to day 15, and short-sell a share from day 16. How can you constrcut such an indicator in the dataset? 

Step 1: Calculate shares of holding for each day

In [15]:
#Calculate shares
data1['Share'] = 0
shares = 0
buy_shares = 1
for index, row in data1.iterrows():
    if row['Buy_Signal']:
        shares += 1 
    elif row['Sell_Signal']:
        if shares > 0:
            shares -= 1
        else:
            shares = 0
    # Update the 'Shares' column in the DataFrame
    data1.at[index, 'Share'] = shares
conditions=[(data1['Buy_Signal'] == True),
            (data1['Sell_Signal'] == True)]
results=[1, -1]
data1['Indicator'] = np.select(conditions, results, default=0)
data1.tail(10)

,Open,High,Low,Close,Adj Close,Volume,90_Day_Rolling,20_Day_Mean,Buy_1,Buy_2,Buy_Signal,Sell_1,Sell_2,Sell_Signal,Daily_Return,Share,Indicator
Date,,,,,,,,,,,,,,,,,
2023-12-29,139.070007,139.699997,138.360001,139.029999,139.029999,1014400,10.737813,134.451000,False,True,False,False,False,False,-0.005294,2,0
2024-01-02,138.190002,140.589996,137.910004,138.750000,138.750000,1441600,10.959337,134.949000,False,True,False,False,False,False,-0.002014,2,0
2024-01-03,138.000000,138.000000,131.070007,131.160004,131.160004,2074500,11.044108,135.063000,False,True,False,False,False,False,-0.054703,2,0
2024-01-04,130.550003,131.500000,130.190002,131.000000,131.000000,2446600,11.124432,135.219000,False,False,False,False,True,False,-0.001220,2,0
2024-01-05,130.000000,131.960007,128.619995,130.559998,130.559998,1394000,11.197034,135.301500,False,False,False,False,True,False,-0.003359,2,0
2024-01-08,130.139999,133.570007,129.809998,133.380005,133.380005,1311400,11.300471,135.536501,False,False,False,False,True,False,0.021599,2,0
2024-01-09,132.270004,135.649994,130.009995,130.679993,130.679993,1435000,11.363833,135.710500,False,False,False,False,True,False,-0.020243,2,0
2024-01-10,130.580002,131.160004,128.360001,131.089996,131.089996,1326300,11.435809,135.816500,False,False,False,False,True,False,0.003137,2,0
2024-01-11,130.580002,130.679993,127.900002,129.679993,129.679993,2060500,11.488396,135.861000,False,False,False,False,True,False,-0.010756,2,0


In [ ]:
# Verify. The new column is "Share." (verification is an important step to check whether your previous implementation is correct.)
df[['Pct Change', 'BUY', 'SELL', 'Share']][4218:4222]

Step 2: Calculate strategic returns according to your holdings of shares and daily stock returns

In [16]:
#create a strategy return series by using the daily stock returns mutliplied by 1 if we are long and -1 if we are short
data1['Total_Return'] = data1['Daily_Return']*data1['Share']
data1['Return'] = data1['Total_Return'].cumsum()
data1.tail(5)

,Open,High,Low,Close,Adj Close,Volume,90_Day_Rolling,20_Day_Mean,Buy_1,Buy_2,Buy_Signal,Sell_1,Sell_2,Sell_Signal,Daily_Return,Share,Indicator,Total_Return,Return
Date,,,,,,,,,,,,,,,,,,,
2024-01-08,130.139999,133.570007,129.809998,133.380005,133.380005,1311400,11.300471,135.536501,False,False,False,False,True,False,0.021599,2,0,0.043199,0.272890
2024-01-09,132.270004,135.649994,130.009995,130.679993,130.679993,1435000,11.363833,135.710500,False,False,False,False,True,False,-0.020243,2,0,-0.040486,0.232404
2024-01-10,130.580002,131.160004,128.360001,131.089996,131.089996,1326300,11.435809,135.816500,False,False,False,False,True,False,0.003137,2,0,0.006275,0.238679
2024-01-11,130.580002,130.679993,127.900002,129.679993,129.679993,2060500,11.488396,135.861000,False,False,False,False,True,False,-0.010756,2,0,-0.021512,0.217167
2024-01-12,130.309998,131.610001,129.639999,130.539993,130.539993,1285200,11.553304,135.700999,False,False,False,False,True,False,0.006632,2,0,0.013263,0.230430


In [ ]:
# Verify again
df[['Pct Change', 'BUY', 'SELL', 'Share', 'Rets']][4218:4222]

#### Good Job! Apply the strategy to all stocks in stocks_list

In [17]:
#create empty list to hold our return series DataFrame for each stock
frames = pd.DataFrame()
 
# for stock in stocks_list:
# Redifine the loop based on my list
for stock in df['Symbol']:
    
    try:
        s_df = yf.download(stock, start='2018-01-01')
        if s_df.empty:
            print(f"No data for ticker {stock}, skipping.")
            continue
        s_df.fillna(0, inplace=True)
        
        s_df['Symbol'] = stock
        s_df['90_Day_Rolling'] = s_df['Close'].rolling(90).std()
        s_df['20_Day_Mean'] = s_df['Close'].rolling(20).mean()

        s_df['Buy 1'] = (s_df['Low'].shift(-1) - s_df['Open']) > (s_df['90_Day_Rolling']/2)
        s_df['Buy 2'] = s_df['Open'] > s_df['20_Day_Mean']
        s_df['Buy'] = s_df['Buy 1'] & s_df['Buy 2']

        s_df['Sell 1'] = (s_df['High'].shift(-1) - s_df['Open']) > (s_df['90_Day_Rolling']/2)
        s_df['Sell 2'] = s_df['Open'] < s_df['20_Day_Mean']
        s_df['Sell'] = s_df['Sell 1'] & s_df['Sell 2']

        s_df['Daily_Return'] = s_df['Close'].pct_change()

        s_df['Share'] = 0
        shares = 0
        for index, row in data1.iterrows():
            if row['Buy_Signal']:
                shares += 1 
            elif row['Sell_Signal']:
                if shares > 0:
                    shares -= 1
                else:
                    shares = 0
            # Update the 'Shares' column in the DataFrame
            s_df.at[index, 'Share'] = shares

        s_df['Total_Return'] = s_df['Daily_Return']*s_df['Share']
        s_df['Return'] = s_df['Total_Return'].cumsum()
        
        ### Copy Paste Previous Code For One Stock ###
                            
        #append the strategy return series to our list

        last_row = s_df.iloc[-1].to_dict()
        #last_row['Symbol'] = stock
        frames = frames.append(last_row, ignore_index=True)
              
    except:
        pass

df = df.merge(frames, on='Symbol', how='left')

[*********************100%%**********************]  1 of 1 completed
/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ignore_index=True)
[*********************100%%**********************]  1 of 1 completed
/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ignore_index=True)
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAC']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed


No data for ticker AAC, skipping.


/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ignore_index=True)
/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  frames = frames.append(last_row, ignore_index=True)
[*********************100%%**********************]  1 of 1 completed
/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ign

No data for ticker AAV, skipping.


[*********************100%%**********************]  1 of 1 completed
/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ignore_index=True)
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ABB']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed


...

No data for ticker ZNH, skipping.


/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ignore_index=True)
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZPIN']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-01-01 -> 2024-01-12)')


No data for ticker ZPIN, skipping.


[*********************100%%**********************]  1 of 1 completed
/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ignore_index=True)
[*********************100%%**********************]  1 of 1 completed
/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ignore_index=True)
[*********************100%%**********************]  1 of 1 completed
/var/folders/vv/3nnd1g4506z6vdqnf44fkr2c0000gn/T/ipykernel_9870/3172015304.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frames = frames.append(last_row, ignor

No data for ticker ZX, skipping.


In [18]:
#df = df.iloc[:10]
df = df.dropna()
df
#frames

,Symbol,Description,Open,High,Low,Close,Adj Close,Volume,90_Day_Rolling,20_Day_Mean,Buy 1,Buy 2,Buy,Sell 1,Sell 2,Sell,Daily_Return,Share,Total_Return,Return
0,A,Agilent Technologies,130.309998,131.610001,129.639999,130.539993,130.539993,1285200.0,11.553304,135.700999,0.0,0.0,0.0,0.0,1.0,0.0,0.006632,2.0,0.013263,0.230430
1,AA,Alcoa Corporation,31.190001,31.820000,30.020000,30.030001,30.030001,3933500.0,2.834549,32.095000,0.0,0.0,0.0,0.0,1.0,0.0,-0.027526,2.0,-0.055052,0.676650
4,AAP,Advance Auto Parts Inc,62.810001,64.220001,61.299999,61.299999,61.299999,1238800.0,4.388411,61.378500,0.0,1.0,0.0,0.0,0.0,0.0,-0.018100,2.0,-0.036201,-0.025669
5,AAT,American Assets Trust,23.850000,23.850000,23.260000,23.430000,23.430000,153800.0,1.831402,22.928500,0.0,1.0,0.0,0.0,0.0,0.0,-0.000853,2.0,-0.001706,0.103063
7,AB,Alliancebernstein Holding L.P.,33.000000,33.279999,32.799999,33.189999,33.189999,402900.0,1.686278,31.627000,0.0,1.0,0.0,0.0,0.0,0.0,0.010658,2.0,0.021315,0.175903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,YUMC,Yum China Holdings Inc,38.000000,38.360001,37.220001,37.480000,37.480000,3313100.0,6.174436,40.751000,0.0,0.0,0.0,0.0,1.0,0.0,-0.016531,2.0,-0.033062,-0.357688
3148,ZBH,Zimmer Biomet Holdings,123.690002,124.750000,122.349998,123.070000,123.070000,1735800.0,6.519344,120.832000,0.0,1.0,0.0,0.0,0.0,0.0,0.005638,2.0,0.011276,-0.304380
3155,ZTO,Zto Express [Cayman] Inc,18.969999,19.165001,18.965000,19.080000,19.080000,2534900.0,1.794661,20.048500,0.0,0.0,0.0,0.0,1.0,0.0,0.005799,2.0,0.011597,0.891079
3156,ZTR,Zweig Total Return Fund,5.500000,5.550000,5.490000,5.490000,5.490000,166000.0,0.306125,5.513500,0.0,0.0,0.0,0.0,1.0,0.0,-0.005435,2.0,-0.010870,-0.343835


#### Plot cummulative returns of strategy 
Step 1: Calculate cumulative returns 

In [ ]:
#concatenate the individual DataFrames held in our list- and do it along the column axis


In [ ]:
#create a column to hold the sum of all the individual daily strategy returns


In [ ]:
#fill 'NaNs' with zeros to allow our "count" function below to work properly


In [ ]:
#create a column that hold the count of the number of stocks that were traded each day
#we minus one from it so that we dont count the "Total" column we added as a trade.


In [ ]:
#create a column that divides the "total" strategy return each day by the number of stocks traded that day to get equally weighted return.


Step 2: Plot 

In [ ]:
#plot the strategy returns


Step 3: Calculate annual returns and sharpe ratio for your strategy

In [25]:
# Annual Return 
daily_return = df['Return'].sum()
# Assume 360 days
annual_return = daily_return*360
# Calculate 90 Day std.
std = df['90_Day_Rolling'].mean()
annual_std = std*np.sqrt(4)
print('Annual Return: ', annual_return)
print('Annual Deviation:', annual_std)

Annual Return:  5171.824702791484
Annual Deviation: 10.02578101235571


In [27]:
# Sharpe Ratio (risk free element excluded for simplicity)
sharpe = annual_return/annual_std
print('Sharpe:', sharpe)

Sharpe: 515.8525501821514


#### Congraduation!!! You have written the first backtesting code yourself!!! 